In [1]:
import nltk
import spacy
import pandas as pd
from tqdm import tqdm
import os
import nltk
import numpy as np
from pattern.text.en import singularize
import nltk
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
import pickle
from collections import Counter

In [7]:
ent_df = pd.read_csv("Litcovid_treatment_ents.csv")

p_tags = ent_df[ent_df.type == "Participant"].text.unique()
p_tags.sort()
p_tags

array(['% African', '% vs', '% vs 0', ..., 'yr', 'zoonotic CoVs',
       'zoonotic viruses'], dtype=object)

In [9]:
pmids = ent_df.pmid.unique()
sim_df = pd.DataFrame(np.zeros((len(pmids), len(p_tags))), columns=p_tags, index=pmids)
sim_df

,% African,% vs,% vs 0,',"'' ,","'' , ``","'' `` coronavirus ,",'classical ' KD,'difficult-to-treat ','low,...,young monkeys,young obese COVID-19,younger,younger ( nonelderly ),younger COVID-19,younger adults,youth,yr,zoonotic CoVs,zoonotic viruses
31981224,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
31996494,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
32013309,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
32015560,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
32017984,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
32022276,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
32022370,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
32023681,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
32023682,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
32023684,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
for pmid in tqdm(ent_df.pmid.unique()):
    ents = ent_df[(ent_df.type == "Participant") & (ent_df.pmid == pmid)].text.unique()
    for ent in ents:
        sim_df.loc[pmid, ent] = sim_df.loc[pmid, ent] + 1

100%|██████████████████████████████████████████████████████████████████████████████| 6002/6002 [01:20<00:00, 74.91it/s]


4.0

In [19]:
def jaccard_similarity(x,y):
    intersection_cardinality = len(set.intersection(*[set(x), set(y)]))
    union_cardinality = len(set.union(*[set(x), set(y)]))
    return intersection_cardinality/float(union_cardinality)

def square_rooted(x):
    return round(sqrt(sum([a*a for a in x])),3)
  
def cosine_similarity(x,y):
    numerator = sum(a*b for a,b in zip(x,y))
    denominator = square_rooted(x)*square_rooted(y)
    return round(numerator/float(denominator),3)
  

% African                                                  2.0
life expectancy                                            2.0
life critically ill                                        2.0
levels > 1200                                              2.0
levels (                                                   2.0
level of co-morbidity                                      2.0
level of ALT                                               2.0
leukoencephalopathy                                        2.0
leukocyte count                                            2.0
leukaemium                                                 2.0
leucopenium                                                2.0
lethal pneumonia                                           2.0
lethal challenge                                           2.0
less than                                                  2.0
less symptomatic                                           2.0
less severe disease                                    

In [20]:
ent_df.pmid.unique().shape

(6002,)